#### Install required packages

In [16]:
# !pip3 install transformers torch spacy pandas
!python3 -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstalled en-core-web-sm-3.7.1

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import spacy
print("spaCy version:", spacy.__version__)

spaCy version: 3.8.7


#### Import libraries

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import spacy
import pandas as pd
import re

#### Load FinBERT model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
labels = ["negative", "neutral", "positive"]


#### Load spaCy NER model (for country/company/sector names)

In [4]:
nlp = spacy.load("en_core_web_sm")


#### Example input (news headline)

In [5]:
headline = "Trump announces 10% tariff on Chinese electronics to protect U.S. industry."


#### Extract entities and event keywords

In [6]:
def extract_info(text):
    doc = nlp(text)
    countries = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "ORG"]]
    # Basic event keyword matching
    if re.search(r"tariff|sanction|ban|restriction|conflict", text, re.I):
        event_type = "Trade Restriction"
    else:
        event_type = "Other"
    return countries, event_type

countries, event_type = extract_info(headline)
print("Entities:", countries)
print("Event Type:", event_type)


Entities: ['Trump', 'U.S.']
Event Type: Trade Restriction


#### Use FinBERT for sentiment Analysis

In [7]:
inputs = tokenizer(headline, return_tensors="pt", truncation=True)
outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
sentiment = labels[probs.argmax()]
impact_score = probs[0][0].item()  # use "negative" prob as proxy for risk
print("Sentiment:", sentiment)
print("Impact Score (risk proxy):", round(impact_score, 3))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment: negative
Impact Score (risk proxy): 0.999


#### Compute a Trade Risk Score

In [8]:
def compute_trade_risk(event_type, impact_score, exposure_weight=0.6):
    base = 0.5 if event_type == "Trade Restriction" else 0.2
    risk = min(1.0, base + impact_score * exposure_weight)
    return round(risk * 100, 1)

risk_score = compute_trade_risk(event_type, impact_score)
print("Predicted Trade Risk Score:", risk_score)


Predicted Trade Risk Score: 100.0


#### Combine results into a DataFrame

In [9]:
data = {
    "Headline": [headline],
    "Entities": [countries],
    "Event Type": [event_type],
    "Sentiment": [sentiment],
    "Impact Score": [impact_score],
    "Trade Risk (%)": [risk_score]
}
df = pd.DataFrame(data)
print(df)


                                            Headline       Entities  \
0  Trump announces 10% tariff on Chinese electron...  [Trump, U.S.]   

          Event Type Sentiment  Impact Score  Trade Risk (%)  
0  Trade Restriction  negative      0.999207           100.0  
